#### latitude and longitude informations in the csv are obtained with the geocoder opensource geocoding api.

In [ ]:
'''import geocoder

# Function to get latitude and longitude
def get_lat_lng(university_name):
    g = geocoder.osm(university_name)
    if g.ok:
        return g.latlng
    else:
        return None

# List of university names
university_names =  masters_ranked['universityName'].unique().tolist()
# Dictionary comprehension to create the desired dictionary
location_dict = {name: get_lat_lng(name) for name in university_names}

'''


#

In [3]:
import pandas as pd
df = pd.read_csv('master_ranked_lat_lot.csv',index_col = 0).reset_index(drop=True)
df

,courseName,universityName,description,fees,location,url,score,latitude,longitude
0,International Management - MSc,Audencia Business School,"About the CourseCUSTOMISABLE, INTERNATIONAL, F...",0.0,Nantes,https://www.findamasters.com/masters-degrees/c...,3.3,47.247009,-1.552633
1,Educational Leadership and Management (Distanc...,University of Portsmouth,"OverviewIf you're currently working, or aspire...",0.0,Portsmouth,https://www.findamasters.com/masters-degrees/c...,2.7,50.795245,-1.093553
2,International Business Management - MSc/PGDip/...,"St Mary’s University, Twickenham",This degree focuses on the development of the ...,0.0,London,https://www.findamasters.com/masters-degrees/c...,2.7,51.437068,-0.336095
3,Educational Leadership and Management MSc,University of Portsmouth,"OverviewIf you're currently working, or aspire...",0.0,Portsmouth,https://www.findamasters.com/masters-degrees/c...,2.7,50.795245,-1.093553
4,Clinical Education MSc/PgDip/PgCert,University of Exeter,OverviewLearn the craft of teaching from inter...,0.0,Exeter,https://www.findamasters.com/masters-degrees/c...,2.7,50.736937,-3.534735
...,...,...,...,...,...,...,...,...,...
5985,Design for Food (Specializing Master),"POLI.design, Società consortile a responsabili...",The Specializing Master DESIGN FOR FOOD integr...,16500.0,Milan,https://www.findamasters.com/masters-degrees/c...,0.3,45.464194,9.189635
5986,Design for Architecture (Specializing Master),"POLI.design, Società consortile a responsabili...","Future designers, architects and engineers wil...",12000.0,Milan,https://www.findamasters.com/masters-degrees/c...,0.3,45.464194,9.189635
5987,Cyber Security (MSc),University of Rennes 1,Cybersecurity has been called the greatest thr...,0.0,Rennes,https://www.findamasters.com/masters-degrees/c...,0.3,48.111339,-1.680020
5988,Econometrics (MSc),Erasmus School of Economics,Would you like to acquire the skills to be abl...,16100.0,Rotterdam,https://www.findamasters.com/masters-degrees/c...,0.3,51.924442,4.477750


In [19]:
import folium
import pandas as pd
import numpy as np


#legend rules
def get_color(fee):
    if fee == 0:
        return 'green'
    elif fee <= 5000:
        return 'blue'
    elif fee <= 10000:
        return 'orange'
    elif fee <= 20000:
        return 'red'
    else:
        return 'purple'

#prevent overlapping locations
def add_jitter(lat, lon, degree_of_jitter=0.001):
    return (lat + np.random.uniform(-degree_of_jitter, degree_of_jitter),
            lon + np.random.uniform(-degree_of_jitter, degree_of_jitter))

#creating map object.
map_center_lat = df['latitude'].mean()
map_center_long = df['longitude'].mean()
university_map = folium.Map(location=[map_center_lat, map_center_long], zoom_start=2)

#putting points on the map
for _, row in df.iterrows():
    jittered_lat, jittered_lon = add_jitter(row['latitude'], row['longitude'])
    
    folium.CircleMarker(
        location=[jittered_lat, jittered_lon],
        radius=5,
        popup=f"{row['universityName']} - {row['courseName']} - Fees: {row['fees']}",
        color=get_color(row['fees']),
        fill=True,
        fill_color=get_color(row['fees'])
    ).add_to(university_map)


#adding legend
legend_html = '''
<div style="position: fixed; 
     top: 50px; right: 50px; width: 200px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background: white; padding: 10px; border-radius: 5px;
     box-shadow: 0 0 15px rgba(0,0,0,0.4);">
     &nbsp; Cost Legend <br>
     &nbsp; Free/No Cost Found &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i><br>
     &nbsp; ≤ $5,000 &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i><br>
     &nbsp; ≤ $10,000 &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i><br>
     &nbsp; ≤ $20,000 &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i><br>
     &nbsp; > $20,000 &nbsp; <i class="fa fa-map-marker fa-2x" style="color:purple"></i>
</div>
'''
university_map.get_root().html.add_child(folium.Element(legend_html))

#saving map as html
university_map.save('university_map.html')
